# Interpretable Active Learning - LIME

####  Uczenie Maszynowe 2  

Autorzy:
- Wojciech Rębisz 
- Marcin Hajdo

_Udemy finance_ dataset: https://drive.google.com/file/d/1kLlzaPng5DsLbjIoBnQTPyuonNoOhUv2/view

Repozytorium LIME: https://github.com/marcotcr/lime

## Krótki opis LIME

Celem narzędzia LIME, jest wytłumaczenie dlaczego model uczenia maszynowego dokonał określonej prognozy. 

LIME testuje, co dzieje się z prognozami, gdy wprowadzasz różne (odmienne) dane do modelu. 

LIME generuje nowy zestaw danych składający się z permutowanych próbek i odpowiednich prognoz modelu (traktowanego jako _black-box_).

Instalacja lime:

In [ ]:
#!pip install lime
#!pip install sklearn

Dodatkowe biblioteki:

In [2]:
import pandas as pd
import numpy as np
from lime import lime_tabular

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

### Wczytanie danych _Udemy Finance_

In [3]:
dataset = pd.read_csv("udemy_finance.csv")

Zobaczmy jakiego rozmiaru są dane, i z czego się składają:

In [4]:
dataset.iloc[0]

id                                                                         762616
title                           The Complete SQL Bootcamp 2020: Go from Zero t...
url                                            /course/the-complete-sql-bootcamp/
is_paid                                                                      True
num_subscribers                                                            295509
avg_rating                                                                4.66019
avg_rating_recent                                                         4.67874
rating                                                                    4.67874
num_reviews                                                                 78006
is_wishlisted                                                               False
num_published_lectures                                                         84
num_published_practice_tests                                                    0
created         

In [5]:
dataset.shape

(13608, 20)

## Przygotowywanie danych

Przekształcamy kolumny _is_paid_ oraz  _is_wishlisted_ z typu _boolean_ na typ _integer_, a także usuwamy puste bloki w datasecie

In [6]:
dataset["is_paid"] = dataset["is_paid"].astype(int)
dataset["is_wishlisted"] = dataset["is_wishlisted"].astype(int)
udemy_df = dataset.dropna()

Wybieramy, którymi kolumnami chcemy się zająć:
- rating = oceny kursu
- is_paid = czy jest płatny
- num_subscribers = liczba subskrybentów
- num_reviews = liczba wyświetleń
- num_published_lectures = liczba opublikowanych zajęć/sesjii
- num_published_practice_tests = liczba testów praktycznych


In [7]:
y = udemy_df["rating"]
X = udemy_df[["is_paid",
            "num_subscribers", 
            "num_reviews", 
            "num_published_lectures",
            "num_published_practice_tests"]]

In [8]:
X[0:2]

is_paid  num_subscribers  num_reviews  num_published_lectures  \
0        1           295509        78006                      84   
1        1           209070        54581                      78   

   num_published_practice_tests  
0                             0  
1                             0

Dzielimy zbiór na treningowy i testowy żeby przewidywać (w tym konkretnym przypadku) wartość _rating_.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1502)

Do estymacjii wykorzystany zostanie _RandomForestRegressor_: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Gdzie liczba drzew w lesie wynosi 200, a randomowy stan początkowy jest równy 1998

In [10]:
model = RandomForestRegressor(random_state=1998, n_estimators=200)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=1998)

_LimeTabularExplainer_ wyjaśnia przewidywania na danych z tabeli (tj. macierzowych) -  skaluje/uśrednia dane.

In [11]:
explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train), 
                                                mode="regression", 
                                                feature_names=X_train.columns, 
                                                categorical_features=[0])

In [12]:
X_test.iloc[4]

is_paid                             1
num_subscribers                 13153
num_reviews                       137
num_published_lectures             10
num_published_practice_tests        0
Name: 2123, dtype: int64

Sprawdźmy co miało największy wpływ na wyniki przewidywań

In [13]:
exp = explainer.explain_instance(data_row=X_test.iloc[4], predict_fn=model.predict)

exp.show_in_notebook(show_table=True)

C:\Python39\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Można dodatkowo wyświetlić jako listę:

In [14]:
exp.as_list()

[('num_reviews > 86.00', 0.44596388380630675),
 ('num_subscribers > 2273.50', -0.15721650652357946),
 ('num_published_lectures <= 12.00', -0.14423989335228196),
 ('num_published_practice_tests <= 0.00', 0.02573650962350094),
 ('is_paid=1', 0.0)]

# Zadanie 1

Sprawdź który z czynników:
* discount_price__amount
* num_published_practice_tests
* is_wishlisted
* num_published_lectures
* price_detail__amount 

miał największy wpływ na ilość recenzjii/opinii *_num_reviews_* w przypadku kursu "C++ Programming for Absolute Beginners. Newbie C++ Guide"

In [15]:
dataset.iloc[746].title

'C++ Programming for Absolute Beginners. Newbie C++ Guide'

In [ ]:
y1 = udemy_df["num_reviews"]
X1 = udemy_df[["discount_price__amount",
            "num_published_practice_tests", 
            "rating", 
            "num_published_lectures",
            "price_detail__amount" ]]


X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=250)

model = RandomForestRegressor(random_state=250, n_estimators=200)
model.fit(X1_train, y1_train)

explainer = # TODO zdefiniuj explainer

print(udemy_df.iloc[...])   # TODO wybierz wiersz, jednak pamiętaj, że struktura datasetu się zmieniła
exp = explainer.explain_instance(data_row=X1.iloc[...], predict_fn=model.predict)

exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
print("Wartość przewidywana:", exp.predicted_value)

In [ ]:
print("Wartość rzeczywista:", y1.iloc[710])

# Zadanie 2

Inne klasyfikatory/regresory: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble

Z podanego linku należy wybrać inny regresor i przetestować czy struktura explainera się zmieniła.

In [ ]:
from sklearn.ensemble import #TODO Wybierz regressor
y2 = udemy_df[] # TODO wybierz poprawną cechę
X2 = udemy_df[[ ]] # TODO Wybierz kolumny


X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=250)

model = # TODO Wykorzystując link wybierz inny regresor
model.fit(X2_train, y2_train)

explainer = # TODO

exp = explainer.explain_instance(data_row=X2.iloc[...], predict_fn=model.predict) # TODO

exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
print("Wartość przewidywana:", exp.predicted_value)

In [ ]:
print("Wartość rzeczywista:", y2.iloc[...]) # TODO

# Zadanie 3

W tym zadaniu celem jest zademonstrowanie działania LIME w przypadku klasyfikacji. Należy więc wybrać ze strony podanej we wcześniejszym zadaniu przykładowy klasyfikator i dokonać klasyfikacji dla zbioru _titanic_. W tym przypadku uwzględniony będzie cały zbiór, bez wyboru predyktorów.

In [ ]:
titanic = pd.read_csv("titanic.csv", sep=';')
titanic.shape
titanic["pclass"] = titanic["pclass"].astype('float64')
titanic["age"] = titanic["age"].astype('float64')
titanic["sibsp"] = titanic["sibsp"].astype('float64')
titanic["parch"] = titanic["parch"].astype('float64')
dataset = titanic.dropna()

In [ ]:
from sklearn.ensemble import # TODO Wybierz klasyfikator

y3 = dataset["survived"]
X3 = dataset[["pclass",
            "sex", 
            "age", 
            "sibsp",
            "parch" ]]


X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=250)

model = #TODO Wybierz klasyfikator
model.fit(X3_train, y3_train)

print("Test  Accuracy : %.2f"%model.score(X3_test, y3_test))
print("Train Accuracy : %.2f"%model.score(X3_train, y3_train))

explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X3_train), 
                                                mode="classification",
                                                feature_names=X3_train.columns,
                                                class_names=["0", "1"],
                                                categorical_features=[0]
                                             )

# TODO wybierz wiersz
exp = explainer.explain_instance(X3.iloc[...], model.predict_proba, num_features=len(X3_train.columns), top_labels=1)

exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()


In [ ]:
print("Predykcja lokalna: ", exp.local_pred)
print("Predykcja globalna: ", exp.predict_proba)